# Algorithm validation using a synthesis image
The synthesis image is the sum of :
- a background extracted from a confocal image;
- sinusoides mimiking the saw marks;
- of cylinders mimiking the features.

$$\begin{array}{l}
im\_synth({x_i},{y_j}) = background({x_i},{y_j}) + \sum\limits_h {{A_h}\sin ({f_h}{y_j}) + \sum\limits_f {{\mathop{\rm circle}\nolimits} ({x_i},{y_j},{x_{0,f}},{y_{0,f}},{r_f}} } ,{h_f})\\
{\mathop{\rm circle}\nolimits} ({x_i},{y_j},{x_{0,f}},{y_{0,f}},{r_f},{h_f}) = \left\{ {\begin{array}{*{20}{c}}
{ - {h_f}\hspace{0.5cm}{\rm{ if }}\hspace{0.5cm}\sqrt {{{\left( {{x_{0,f}} - {x_i}} \right)}^2} + {{\left( {{y_{0,f}} - {y_j}} \right)}^2}}  \le {r_f}}\\
{0\hspace{0.5cm}{\rm{ if }}\hspace{0.5cm}\sqrt {{{\left( {{x_{0,f}} - {x_i}} \right)}^2} + {{\left( {{y_{0,f}} - {y_j}} \right)}^2}}  > {r_f}}
\end{array}} \right.
\end{array}$$  

In [ ]:
 
# Standard library imports
import os.path
from pathlib import Path

# Path of 'site-packages' where useful packages are stored on MAC-OS
mac_packages = "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages"    
    
# Root directory for all files 
root = Path("C:/Users/franc/OneDrive/Bureau/confocal/fichiers")
if not os.path.isdir(root) : 
    root = Path('/Users/amal/Gwyddion Images /Carton-Louise') 
store_file = root / Path('synthesis.xlsx')

# Parameters of image reference 
cut_number = '119'
wafer_number = '2'
image_ref = 'c'

# Specification of the directories of the experimental files and files names
root_input = root / Path('cut ' + cut_number) / Path('wafer ' + wafer_number) / Path('save_plu')
input_file_name = Path('wafer' + wafer_number +'-' + image_ref +'.plu') # input file 

# Specification of the directories of the output files and files names
root_output = root / Path('cut ' + cut_number) / Path('wafer ' + wafer_number)
file_xlsx_th = Path('im_synth.xlsx')      # file EXEL features features sythezed image
file_xlsx_res = Path('im_synth_res.xlsx') # file EXEL features morphological analysis



In [ ]:
'''
This cell generates an image (N rows by M columns), with cylindrical features of radius r; depth h
at x0, y0 postion in µm. A background extacted from an experimental image is added
The features can be generated both by a determinitic way or by a random procedure.

'''

# 3rd party imports
import numpy as np
import pandas as pd

# Add package image_features_extract
try: # standard storage path of 'site-packages' on WIN
    import image_features_extract as ife
except: # Add storage path of 'site-packages' on MAC-OS
    import sys
    sys.path.append(mac_packages)
    import image_features_extract as ife
    
test = 0 #  deterministic features generation test =1
         # random features generation test = 0

#  description of features for deterministic generation (x0, y0, r, h) in µm
features = [(50,50,5,1),(80,50,5,1),(100,50,5,1),(150,50,5,1),(200,50,5,1),
               (200,350,20,1.5),(200,200,15,1.4),(500,500,20,1.2)]

#  Number of features for random generation 
random_feature = {}
random_feature['n_indent'] = 20 # number of indentation features
random_feature['n_small'] = 5 # small features
random_feature['n_cleavage'] = 20  # cleavage features generation


cir = lambda X,Y : np.where((np.sqrt((X-x0)**2+(Y-y0)**2)-r) < 0, h,0.0)


# background determination
file = root_input / input_file_name
N,M,conf_img,_ = ife.read_plu_topography(file) # fle .plu reading
conf_img = ife.fill_gap(conf_img)              # substitute non measured values by interpolated ones 

im_flat = ife.image_flattening(conf_img, method_flat='poly',kx=5,ky=5) # image background removal
im_synth = conf_img - im_flat                                          # image background extraction

x = np.array(range(M))
y = np.array(range(N))

# saw marks generation
sines = [(0.1,200),(0.05,50),(0.2,20)] 
for sine in sines:
    A,f = sine
    im_synth = im_synth+ A*np.sin(f*y/N).reshape((N,1))

dict_feature = {}
X,Y = np.meshgrid(x,y)

# features generation
if test==1: # deterministic features generation

    for idx,feature in enumerate(features):
        x0, y0, r, h = feature
        c = cir(X,Y)
        dict_feature[idx+1] = [x0,y0,r,h,len(np.where(c==h)[0])]
        im_synth = im_synth - c
        
else:   # random features generation
    n_features = random_feature['n_indent'] # number of indentation features
    num_feature=1   # feature index
    for idx in range(n_features):
        x0 = np.random.randint(1,M) # x location of the feature limited to image width (M pixels)
        if idx%5 ==0 : y0 = np.random.randint(1,N) # force 5 features alignement on a row
                                                   # y location of the feature limited to image height (N pixels)
        r = np.random.randint(1,5) # radius range limited to 1 - 5 µm
        h = 0.5 # uniform indent depth may be randomized using np.random.randint(1,200)*0.01
        c = cir(X,Y)
        dict_feature[num_feature] = [x0,y0,r,h,len(np.where(c==h)[0])]
        im_synth = im_synth - c
        num_feature += 1

    n_features = random_feature['n_small'] # small features
    for idx in range(n_features):
        x0 = np.random.randint(1,M)  # x location of the feature limited to image width (M pixels)
        y0 = np.random.randint(1,N)  # y location of the feature limited to image height (N pixels)
        r = np.random.randint(1,5)   # radius range limited to 1 - 5 µm
        h = np.random.randint(1,200)*0.01 # depth range limited to 0.01 - 2 µm
        c = cir(X,Y)
        dict_feature[num_feature] = [x0,y0,r,h,len(np.where(c==h)[0])]
        im_synth = im_synth - c
        num_feature += 1

    n_features = random_feature['n_cleavage']  # cleavage features generation
    for _ in range(n_features):
        x0 = np.random.randint(1,M) # x location of the feature limited to image width (M pixels)
        y0 = np.random.randint(1,N) # y location of the feature limited to image height (N pixels)
        r = np.random.randint(6,20)        # radius range limited to 6 - 20 µm
        h = np.random.randint(50,200)*0.01 # depth range limited to 0.5 - 2 µm
        c = cir(X,Y)
        dict_feature[num_feature] = [x0,y0,r,h,len(np.where(c==h)[0])]
        im_synth = im_synth - c
        num_feature += 1
df_th = pd.DataFrame.from_dict(dict_feature, orient='index',columns=["x","y","r","height","size"])

In [ ]:
'''
Generated image processing:
- saw marks removal and background correction using a top hat filter with a pattern length
larger than the largest feature to extract
- image binarization using a threshold on feature depth larger than the residual image background
- features morphological analysis (using the scipy label function). Generate a
        dataframe  index|'x'|'long_x'| 'y'|'long_y'| 'size'|'depth' where
            - index: feature index
            - x: gravity center x position of the feature
            - long_x: maximum feature width
            - y: gravity center y position of the feature
            - long_y: maximum feature height
            - size: pixels number of the feature 
            - depth : feature depth

Plot of the results (images, histogram, box plot)

Store the results in an excel file

'''

# 3rd party imports
import numpy as np
import matplotlib.colors as colx
import matplotlib.cm as cmx
import matplotlib.pyplot as plt

# Add package image_features_extract
try: # standard storage path of 'site-packages' on WIN
    import image_features_extract as ife
except: # Add storage path of 'site-packages' on MAC-OS
    import sys
    sys.path.append(mac_packages)
    import image_features_extract as ife
    
%matplotlib inline

store_flag = True # if store_flag = True we store the resuts in an excel file

fig = plt.figure(figsize=(15,15))

# parameters initialization
Top_hat_length = 200 #  should be larger than the largest feature to extract
threshold = -0.2     # should be larger than the residual image background

# top hat processing
im_corr = ife.top_hat_flattening(im_synth,Top_hat_length,top_hat_sens=-1)

# binarization
im_bin = np.where(im_corr < threshold, 1, 0) # binarization

df = ife.analyse_morphologique(im_bin,im_corr) # morphological analysis df dataframe

# generated image plot
plt.subplot(3,2,1)
plt.imshow(im_synth, cmap='gray')
plt.colorbar()
plt.title('Raw image')


# top-hat filtered image plot
plt.subplot(3,2,2)
plt.imshow(im_corr, cmap='gray')
plt.colorbar()
plt.title(f'Top hat length: {Top_hat_length}')

# binarized image plot
plt.subplot(3,2,3)
plt.imshow(im_bin, cmap='gray')
plt.colorbar() 
plt.title(f'Threshold: {threshold} µm')

# reconstructed image plot
plt.subplot(3,2,4)
colorsMap = plt.get_cmap("plasma") #("RdYlGn")
g = list(zip(np.array(df['x']),
             np.array(df['y']),
             np.array(df['height']),
             np.array(df['size'])))
x,y,c,s=zip(*sorted(g, key=lambda tup: tup[2],reverse=False))

cm = plt.get_cmap(colorsMap)
cNorm = colx.Normalize(vmin=min(c), vmax=max(c))
scalarMap = cmx.ScalarMappable(norm=cNorm, cmap=cm)
conv_inch_pixel = 5 # sloppy determination must be improved
plt.scatter(np.array(x) ,N-np.array(y) ,s=np.array(s)/conv_inch_pixel, c=scalarMap.to_rgba(c)) # beware image origine the upper leeft corner                                                                          #        plot origine lower left corner
plt.xlim(0,M)
plt.ylim(0,N)
scalarMap.set_array(c)

_ = plt.yticks(ticks=[N,N-100, N-200, N-300, N-400, N-500],labels=[0,100,200,300,400,500],
                               rotation=0)
plt.colorbar(scalarMap)
plt.title("Size and height")

# features size histogram plot
plt.subplot(3,2,5)
h = plt.hist(df['size'], bins=100)
plt.title(f'Size histogram')
plt.ylabel('size (pix\u00B2)')

# features size box plot
plt.subplot(3,2,6)
_ = plt.boxplot(df['size'])
plt.title(f'Size bbox')
_ = plt.ylabel('size (pix\u00B2)')

if store_flag:
    df_th.to_excel(root_output/file_xlsx_th)
    df.to_excel(root_output/file_xlsx_res)

In [ ]:
'''
Profiles plots
'''
# 3rd party imports
import matplotlib.pyplot as plt

%matplotlib inline

fig = plt.figure(figsize=(15,15))
for idx,col in enumerate([480,500,550,570]): #[50,80,150,500,550,570]
    plt.subplot(3,2,idx+1)
    plt.plot(im_synth[col,:],label="raw image" )
    plt.plot(im_corr[col,:],'--',label="top hat" )
    plt.plot([0,M],[threshold,threshold],'k--',label="threshold" )
    plt.legend()
